#### 先找出23-24營業毛利資料的共同公司

In [ ]:
import pandas as pd
sc_24=pd.read_excel("SC_上市櫃(23-24).xlsx")

In [5]:
import pandas as pd
import numpy as np
sc_24=pd.read_excel("SC_上市櫃(23-24).xlsx",sheet_name="Sheet1")
sc_24_overall=pd.read_excel("SC_上市櫃(23-24).xlsx",sheet_name="大盤指數_24")
sc_23=pd.read_excel("SC_上市櫃(23).xlsx",sheet_name="Sheet1")
sc_23_overall=pd.read_excel("SC_上市櫃(23).xlsx",sheet_name="大盤指數_23")
sc_24.loc[:,"代號"]=sc_24.loc[:,"代號"].astype(int)
sc_23.loc[:,"代號"]=sc_23.loc[:,"代號"].astype(int)
sc_24=sc_24.sort_values(by="代號",ascending=True)
sc_23=sc_23.sort_values(by="代號",ascending=True)

with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')

with pd.ExcelWriter('SC_上市櫃(23).xlsx', engine='openpyxl') as writer:
    sc_23.to_excel(writer, index=False,sheet_name='Sheet1')
    sc_23_overall.to_excel(writer, index=False,sheet_name='大盤指數_23')




In [19]:
sc_24=pd.read_excel("SC_上市櫃(23-24).xlsx",sheet_name="Sheet1")
sc_24_overall=pd.read_excel("SC_上市櫃(23-24).xlsx",sheet_name="大盤指數_24")
sc_23=pd.read_excel("SC_上市櫃(23).xlsx",sheet_name="Sheet1")
sc_23_overall=pd.read_excel("SC_上市櫃(23).xlsx",sheet_name="大盤指數_23")

In [6]:
same_2324,arr_24,arr_23=np.intersect1d(sc_24["代號"].values.flatten(),sc_23["代號"].values.flatten(),return_indices=True)

print(len(sc_24.loc[arr_24,"代號"]))
print(len(sc_23.loc[arr_23,"代號"]))
with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.iloc[arr_24,:].to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')

with pd.ExcelWriter('SC_上市櫃(23).xlsx', engine='openpyxl') as writer:
    sc_23.iloc[arr_23,:].to_excel(writer, index=False,sheet_name='Sheet1')
    sc_23_overall.to_excel(writer, index=False,sheet_name='大盤指數_23')





1929
1929


### 再找出23_24資料與現有上市櫃公司的共同公司

#### <span style = "color : #FA5C5C">發現 : 金融業無營業毛利 </span>
#### <a href="https://ppt.cc/fZ35cx"><img src="https://ppt.cc/fZ35cx@.png" border="0" alt="PPT.cc縮圖服務" title="PPT.cc縮圖服務"></a>


In [44]:
df_fs=pd.read_excel("上市櫃公司.xlsx")
sc_24["營業毛利_23"]=sc_23["營業毛利"]
sc_24=sc_24.fillna(0)
with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')


same_2324,arr_24,df_fs_index=np.intersect1d(sc_24["代號"].values.flatten(),df_fs["代號"].values.flatten(),return_indices=True)

print(len(sc_24.loc[arr_24,"代號"]))
with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.iloc[arr_24,:].to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')



1915


### 將上市櫃公司資料的TSE產業名稱納入 sc_24中

### bug : 不知為何比對下會少 中化控股 ( 遺漏公司 : 1家)

###  <span style="color: red">原因 : 23年資料無此家公司，在執行上忽略了要以23年資料比對25年資料 </span>

#### <span style="color: green"> 解決方式 : 先另外儲存共同企業的產業名稱，再用pd.concat(axis=1)合併2個excel </span>

#### axis=1 : 左右拼接   axis = 0 ，上下拼接


In [ ]:
indus_name=df_fs.loc[df_fs_index,"TSE產業名稱"]
print(indus_name)
sc_24=pd.concat([sc_24,indus_name],axis=1)
with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')


0       水泥工業
1       水泥工業
2       水泥工業
3       水泥工業
4       水泥工業
        ... 
1911    綠能環保
1912    鋼鐵工業
1913    運動休閒
1914    鋼鐵工業
1915    存託憑證
Name: TSE產業名稱, Length: 1915, dtype: object


NameError: name 'sc_24_combine' is not defined